<a href="https://colab.research.google.com/github/nishi1612/CS306-Data-Analysis-and-Visualization/blob/master/201601408_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math
import seaborn as sns
from sklearn import preprocessing
from scipy.stats import norm
from scipy.stats import iqr

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1cdGFghBnmVSV1xF85kZ90Y5cRQN7pWmq'})
download.GetContentFile('test.csv')

In [0]:
df = pd.read_csv('test.csv')

In [0]:
copy_one = df

In [0]:
f = df

In [0]:
to_drop = ["Year", "Month", "DayofMonth", "DayOfWeek", 
           "DepTime", "CRSDepTime", "ArrTime", "CRSArrTime", "UniqueCarrier", 
           "FlightNum", "TailNum", "ActualElapsedTime", "CRSElapsedTime","ArrDelay", "DepDelay", "Origin", "Dest",
           "TaxiIn", "TaxiOut", "Cancelled", "CancellationCode", "Diverted", 
           "CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", 
           "LateAircraftDelay"]
f.drop(to_drop, inplace=True, axis=1)

In [0]:
f = f[np.isfinite(f['AirTime'])]
f = f[np.isfinite(f['Distance'])]

In [0]:
X = f['Distance'].values
Y = f['AirTime'].values

In [0]:
X, Y

(array([810, 810, 515, ..., 545, 533, 874]),
 array([116., 113.,  76., ...,  85.,  89., 104.]))

**Question 1**

In [0]:
def slope_intercept(x,y):
  m = (np.mean(x) * np.mean(y) - np.mean(x*y) ) / (np.mean(x) * np.mean(x) - np.mean(x*x))
  m = round(m,4)
  b = np.mean(y) - (np.mean(x) * m)
  b = round(b,4)
  return b,m

In [0]:
b,m = slope_intercept(X,Y)

In [0]:
print("Intercept : " + str(b))
print("Slope : " + str(m))

Intercept : 18.2454
Slope : 0.1177


In [0]:
y_predicted = m*X + b

In [0]:
rmse = 0
for i in range(len(Y)):
  rmse = rmse + ((Y[i]-y_predicted[i])**2)
rmse = math.sqrt(rmse/len(Y))
print("Rmse : " + str(rmse))

Rmse : 12.427075557562077


**Question 2 : Part 1 : 95% confidence interval for value of slope**

In [0]:
sx = np.sqrt(np.sum((X - np.mean(X))**2))
val = 1.96
upper = m + val*np.std(Y)/sx
lower = m - val*np.std(Y)/sx

lower = round(lower,4)
upper = round(upper,4)

print("Lower bound : " + str(lower))
print("Upper bound : " + str(upper))

Lower bound : 0.1176
Upper bound : 0.1178


**Question 2 : Part 2 : 95% confidence interval for value of y_0**

In [0]:
Xo = 1200
n = len(X)
y_1 = m*Xo + b
s = np.sum((Y - y_predicted)**2)/(n - 2)
k = np.sqrt((1/n) + (Xo - np.mean(X))/(np.sum((X - np.mean(X))**2)))
lower1 = y_1 - s*k
upper1 = y_1 + s*k

lower1 = round(lower1,4)
upper1 = round(upper1,4)
print("Lower bound : " + str(lower1))
print("Upper bound : " + str(upper1))

Lower bound : 159.4264
Upper bound : 159.5444


**Question 3**

In [0]:
weight_y = []
d = Y - y_predicted
s = iqr(d)
for i in range(len(X)):
  u = d[i]/(3*s)
  if(u > 1):
    weight_y.append(0)
  else:
    num = (1 - u**2)**2
    weight_y.append(num)

In [0]:
sum1 = 0
normx = X - np.mean(X)
normy = Y - np.mean(Y)
for i in range(len(weight_y)):
  sum1 += weight_y[i]*normx[i]*normy[i]
  
m1 = sum1/np.dot(weight_y,normx**2)
y_weighted = np.dot(weight_y,Y)/np.sum(weight_y)
x_weighted = np.dot(weight_y,X)/np.sum(weight_y)

b1 = y_weighted - slope*x_weighted

m1 = round(slope,4)
b1 = round(intercept,4)

print("Slope : " + str(m1))
print("Intercept : " + str(b1))

Slope : 0.1158
Intercept : 18.2908


In [0]:
y_predicted_2 = b1 + m1*X

In [0]:
rmse = 0
for i in range(len(Y)):
  rmse = rmse + (y_predicted_2[i] - Y[i])**2

rmse = rmse/len(Y)
rmse = math.sqrt(rmse)
rmse = round(rmse,4)
print("Rmse : " + str(rmse))

Rmse : 12.544
